* https://uniswapv3book.com/index.html
* https://medium.com/@chaisomsri96/defi-math-uniswap-v3-concentrated-liquidity-bd87686b3ecf
* https://liobaheimbach.github.io/assets/pdf/Papers/Risks_and_Returns_of_Uniswap_V3_Liquidity_Providers.pdf
* https://docs.uniswap.org/sdk/v3/guides/advanced/active-liquidity
* https://blog.uniswap.org/uniswap-v3-math-primer
* https://blog.uniswap.org/uniswap-v3-math-primer-2
* https://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf

In [1]:
from uniswappy import *
import pandas as pd
import copy
import matplotlib.pyplot as plt

user_nm = MockAddress().apply()
eth_amount = 10000
tkn_amount = 100000

fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

In [ ]:
n_steps = 50
start_price = eth_amount/tkn_amount
mu = 0.1; sigma = 0.5
n_paths = 1

b = BrownianModel(start_price)
p_arr = b.gen_gbms(mu, sigma, n_steps, n_paths)
exp_p_arr = np.median(p_arr, axis = 1)

accounts = MockAddress().apply(250)

In [ ]:
x_val = np.arange(0,len(p_arr))
fig, (USD_ax) = plt.subplots(nrows=1, sharex=False, sharey=False, figsize=(18, 5))
USD_ax.plot(x_val[1:], p_arr[1:], color = 'r',linestyle = 'dashdot', label='initial invest') 
USD_ax.set_title(f'Price Chart (ETH/TKN)', fontsize=20)
USD_ax.set_ylabel('Price (USD)', size=20)
USD_ax.set_xlabel('Date', size=20)

In [2]:
def calc_Lx(sqrtP, dx, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = sqrtP
    Lx = dx/(1/p_sqrt_human - 1/pb_sqrt_human)
    return Lx

def calc_Ly(sqrtP, dy, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = sqrtP
    Ly = dy/(p_sqrt_human - pa_sqrt_human)
    return Ly

def calc_y(dL, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = lp.slot0.sqrtPriceX96/2**96
    dy = dL*(p_sqrt_human - pa_sqrt_human)
    #y = dL*(pb_sqrt_human - pa_sqrt_human)
    return dy

def calc_x(dL, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = lp.slot0.sqrtPriceX96/2**96
    dx = dL*(1/p_sqrt_human - 1/pb_sqrt_human)
    #x = dL*(1/p_sqrt_human - 1/pb_sqrt_human)
    return dx

In [3]:
def get_reserves(lp, token_in):
    tokens = lp.factory.token_from_exchange[lp.name]
    if(lp.version == UniswapExchangeData.VERSION_V2):
        if(token_in.token_name == lp.token1):
            x = lp.get_reserve(tokens[lp.token0])
            y = lp.get_reserve(tokens[lp.token1])
        else: 
            x = lp.get_reserve(tokens[lp.token1])
            y = lp.get_reserve(tokens[lp.token0])
    elif(lp.version == UniswapExchangeData.VERSION_V3):   
        if(token_in.token_name == lp.token0):
            x = lp.get_reserve(tokens[lp.token0])
            y = lp.get_reserve(tokens[lp.token1])
        else: 
            x = lp.get_reserve(tokens[lp.token1])
            y = lp.get_reserve(tokens[lp.token0]) 
    return (x, y)  

def calc_lp_settlement(lp, token_in, itkn_amt, lwr_tick, upr_tick):

    L = lp.get_liquidity()
    #(x, y) = get_reserves(lp, token_in) 
    x = calc_y(L, lwr_tick, upr_tick)
    y = calc_x(L, lwr_tick, upr_tick)
    #print(f'x {x}, y {y}')
    
    gamma = 997

    a1 = x*y/L
    a2 = L
    a = a1/a2
    b = (1000*itkn_amt*x - itkn_amt*gamma*x + 1000*x*y + x*y*gamma)/(1000*L);
    c = itkn_amt*x;

    dL = (b*a2 - a2*math.sqrt(b*b - 4*a1*c/a2)) / (2*a1);
    return dL

def calc_withdraw_portion(lp, token_in, amt, lwr_tick, upr_tick):

    L = lp.get_liquidity()
    #(x, y) = get_reserves(lp, token_in)
    x = calc_y(L, lwr_tick, upr_tick)
    y = calc_x(L, lwr_tick, upr_tick)    
    gamma = 997/1000

    dL = calc_lp_settlement(lp, token_in, amt, lwr_tick, upr_tick)
    dx = dL*x/L
    dy = dL*y/L
    aswap = (gamma*dx)*(y-dy)/(x-dx+gamma*dx)

    return dy/amt 

def calc_lp_settlement2(lp, token_in, q, lwr_tick, upr_tick):

    L = lp.get_liquidity()
    dPy = calc_y(1, lwr_tick, upr_tick)
    dPx = calc_x(1, lwr_tick, upr_tick)
    (x, y) = get_reserves(lp, token_in)
    
    a = dPx*dPy
    b = (1000*q*dPx - 997*q*dPx + 1000*dPy*x + 997*y*dPx)/(1000);
    c = q*x;

    dL = (b - math.sqrt(b*b - 4*a*c)) / (2*a);
    return dL


In [4]:
q96 = 2**96
def price_to_sqrtp(p):
    return int(math.sqrt(p) * q96)

def tick_to_price(tick):
    return math.floor(1.0001**tick)

def price_to_tick(p):
    return math.floor(math.log(p, 1.0001))

def liquidity0(amount, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return (amount * (pa * pb) / q96) / (pb - pa)

def liquidity1(amount, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return amount * q96 / (pb - pa)

def calc_amount0(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * q96 * (pb - pa) / pa / pb)


def calc_amount1(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * (pb - pa) / q96)

def price_to_tick(p):
    return math.floor(math.log(p, 1.0001))

In [ ]:
sqrtp_low = price_to_sqrtp(4525)
sqrtp_cur = price_to_sqrtp(5000)
sqrtp_upp = price_to_sqrtp(5483)

eth = 10**18
amount_eth = 1 * eth
amount_usdc = 5000 * eth

liq0 = liquidity0(amount_eth, sqrtp_cur, sqrtp_upp)
liq1 = liquidity1(amount_usdc, sqrtp_cur, sqrtp_low)
liq = int(min(liq0, liq1))
liq/10**18

In [ ]:
amount_in = 42 * eth
price_diff = (amount_in * q96) // liq
price_next = sqrtp_cur + price_diff
print("New price:", (price_next / q96) ** 2)
print("New sqrtP:", price_next)
print("New tick:", price_to_tick((price_next / q96) ** 2))

amount_in = calc_amount1(liq, price_next, sqrtp_cur)
amount_out = calc_amount0(liq, price_next, sqrtp_cur)

print("USDC in:", amount_in / eth)
print("ETH out:", amount_out / eth)

In [ ]:
import math

eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")
init_price = UniV3Utils.encodePriceSqrt(5000, 1)

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

#lwr_tick = lp.get_tick_price(-1, 10, 1000)
#upr_tick = lp.get_tick_price(1, 10, 1000)

cur_tick = 85176
lwr_tick = 84222
upr_tick = 86129

cur_tick = cur_tick - cur_tick % lp.tickSpacing
lwr_tick = lwr_tick - lwr_tick % lp.tickSpacing
upr_tick = upr_tick - upr_tick % lp.tickSpacing

#lwr_tick = UniV3Utils.getMinTick(tick_spacing)
#upr_tick = UniV3Utils.getMaxTick(tick_spacing)

sqrtP =  math.sqrt(5000)
Ly = calc_Ly(sqrtP, 100, lwr_tick, upr_tick)
Lx = calc_Lx(sqrtP, 1, lwr_tick, upr_tick)

lp.initialize(init_price)
out = lp.mint(user_nm, lwr_tick, upr_tick, Lx)  
lp.summary()

out = Swap().apply(lp, tkn, user_nm, 42) 
lp.summary()
out[1]

In [32]:
import math

eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")
init_price = UniV3Utils.encodePriceSqrt(10, 1)

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

lwr_tick = lp.get_tick_price(-1, 10, 1000)
upr_tick = lp.get_tick_price(1, 10, 1000)

#lwr_tick = UniV3Utils.getMinTick(tick_spacing)
#upr_tick = UniV3Utils.getMaxTick(tick_spacing)

sqrtP =  math.sqrt(10)
Ly = calc_Ly(sqrtP, 100, lwr_tick, upr_tick)
Lx = calc_Lx(sqrtP, 1000, lwr_tick, upr_tick)

lp.initialize(init_price)
out = lp.mint(user_nm, lwr_tick, upr_tick, Lx)    #3162.2776601683795
lp.summary()

lwr_tick = lp.get_tick_price(-1, lp.get_price(eth), 1000)
upr_tick = lp.get_tick_price(1, lp.get_price(eth), 1000)
#lwr_tick = UniV3Utils.getMinTick(tick_spacing)
#upr_tick = UniV3Utils.getMaxTick(tick_spacing)

amount_out = 100
Lx = calc_Lx(sqrtP, amount_out, lwr_tick, upr_tick)
p_out = calc_withdraw_portion(lp, tkn, amount_out, lwr_tick, upr_tick)
#dL2 = calc_lp_settlement(lp, eth, amount_out, lwr_tick2, upr_tick2)
#dL = calc_lp_settlement2(lp, eth, amount_out, lwr_tick2, upr_tick2)
#print(f'dL {dL2} dL2 {dL}')

p_out = 0.493008
(_, _, _, _, amount0, amount1) = lp.burn(user_nm, lwr_tick, upr_tick, p_out*Lx)
lp.summary()

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 999.9999999999997, TKN = 10340.759310919504
Gross Liquidity: 66597.92378128914 

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 950.6991999999997, TKN = 9830.951604283724
Gross Liquidity: 63314.59286053256 



In [33]:
eth_gwei = 10**18
liq = lp.get_liquidity() * eth_gwei
amount_in = amount1 * eth_gwei
sqrtp_cur = price_to_sqrtp(lp.get_price(eth))

price_diff = ((amount_in - 3*amount_in/1000) * q96) // liq
price_next_calc = sqrtp_cur + price_diff
#price_next = math.sqrt(10.05083685047008)*q96
print("New price (actual):", (price_next / q96) ** 2)
print("New price (calc):", (price_next_calc / q96) ** 2)
print("New sqrtP:", price_next)
print("New tick:", price_to_tick((price_next_calc / q96) ** 2))

amount_in = calc_amount1(liq, price_next_calc, sqrtp_cur)
amount_out = calc_amount0(liq, price_next_calc, sqrtp_cur)

print("USDC in:", amount_in / eth_gwei)
print("ETH out:", amount_out / eth_gwei)

dx2 = amount_out / eth_gwei

New price (actual): 10.05083685047008
New price (calc): 10.05083685047008
New sqrtP: 2.5117747796363857e+29
New tick: 23077
USDC in: 508.2782835158756
ETH out: 50.69912253362368


In [34]:
dL_act = 3283.3309207565794

dPy = calc_y(1, lwr_tick, upr_tick)
dPx = calc_x(1, lwr_tick, upr_tick)

L = lp.get_liquidity()

x = lp.get_reserve(eth)
y = lp.get_reserve(tkn)

dx = dL_act*dPx
dx + dx2

99.99992253362367

In [9]:
out = Swap().apply(lp, tkn, user_nm, amount1) 
lp.summary()
print(f'amount0 {amount0}, amount1 {amount1}')
out[1]

new price 251177477963638562769788518347
Exchange ETH-TKN (LP)
Real Reserves:   ETH = 900.0000774663762, TKN = 10340.759310919504
Gross Liquidity: 63314.59286053256 

amount0 49.30079999999998, amount1 509.8077066357803


-50.69912253362341

In [10]:
(251177477963638562769788518347 / q96) ** 2

10.05083685047008

In [ ]:
Lx = 6659.792378128914
p_needed = 0.49301
dL_act = p_needed*Lx
dL_act

In [ ]:
dL_act = 3283.3309207565794

dPy = calc_y(1, lwr_tick2, upr_tick2)
dPx = calc_x(1, lwr_tick2, upr_tick2)

L = lp.get_liquidity()

x = lp.get_reserve(eth)
y = lp.get_reserve(tkn)

dx = dL_act*dPx
dy = dL_act*dPy

#dx = 49.30099999999998
#dy = 509.8097747876425

#x = 950.6989999999996
#y = 9830.949536131862

#dx = dL_act*x/L
#dy = dL_act*y/L
gamma = 997/1000

amount_out - dx
dy

In [ ]:
L = lp.get_liquidity()

dP = dy/L
sqrtP_curr = lp.slot0.sqrtPriceX96/2**96
sqrtP_target = sqrtP_curr + dP
p_target = sqrtP_target**2
dInvP = 1/sqrtP_target - 1/sqrtP_curr 
dInvP*L

In [ ]:
dP = dy/L
sqrtP_curr = lp.slot0.sqrtPriceX96/2**96
sqrtP_target = sqrtP_curr + dP
p_target = sqrtP_target**2
p_target

In [ ]:
dInvP = 1/sqrtP_target - 1/sqrtP_curr 
dInvP

In [ ]:
dInvP*L

In [ ]:
L*(sqrtP_curr - sqrtP_target)

In [ ]:
L*(sqrtP_curr - sqrtP_target) / (sqrtP_curr*sqrtP_target)

In [ ]:
p_out = 0.493008
(_, _, _, _, amount0, amount1) = lp.burn(user_nm, lwr_tick, upr_tick, p_out*Lx)
lp.summary()

out = Swap().apply(lp, tkn, user_nm, amount1) 
lp.summary()
print(f'amount0 {amount0}, amount1 {amount1}')
out

In [ ]:
dPy = calc_y(1, lwr_tick, upr_tick)
dPx = calc_x(1, lwr_tick, upr_tick)
q = amount_out
gamma = 997/1000
x = lp.get_reserve(eth)
y = lp.get_reserve(tkn)

a = dPx*dPy
b = -q*dPx + q*gamma*dPx - dPy*x - y*dPx*gamma;
c = q*x;

dL = (-b + math.sqrt(b**2 - 4*a*c)) / (2*a);
dL

In [ ]:
(gamma*dy*x - dx)/(y - dy + gamma*dy)

In [ ]:
dL_act*dPy

In [ ]:
(dL**2)*dPx*dPy + (dL)*(-q*dPx + q*gamma*dPx - dPy*x - y*dPx*gamma) + q*x

In [ ]:
(_, _, _, _, amount0, amount1) = lp.burn(user_nm, lwr_tick, upr_tick, p_out*Lx)
amount0

In [ ]:
p_out*Lx

In [ ]:
p_out = 0.49301
#p_out = 1
(_, _, _, _, amount0, amount1) = lp.burn(user_nm, lwr_tick, upr_tick, p_out*Lx)
print(f'amount0 {amount0}, amount1 amount1

lp.summary()

out = Swap().apply(lp, tkn, user_nm, amount1) 
withdrawn = abs(out[1])  + p_out*amount_out 
lp.summary()
withdrawn
p_out

In [ ]:
66597.92378128914 - 63314.5795409478 

In [ ]:
0.5138983378376984*Lx

In [ ]:
lp.get_virtual_reserve(tkn)

In [ ]:
calc_x(2999.7687368341976, lwr_tick, upr_tick)

In [ ]:
lp.get_price(eth)

In [ ]:
lp.get_tick_price(0, lp.get_price(eth), 1000)

In [ ]:
lp.get_virtual_reserve(tkn)

In [ ]:
math.sqrt(1.0001**lwr_tick)

In [ ]:
calc_Ly(1000, lwr_tick, upr_tick)

In [ ]:
p_sqrt_human = lp.slot0.sqrtPriceX96/2**96
p_sqrt_human

In [ ]:
p_sqrt_human = math.sqrt(10)
p_sqrt_human

In [ ]:
p_sqrt_human = math.sqrt(1.0001**mid_tick)
p_sqrt_human**2

In [ ]:
dx = lp.get_liquidity()*(1/p_sqrt_human - 1/pb_sqrt_human)
dx

In [ ]:
dy = lp.get_liquidity()*(p_sqrt_human - pa_sqrt_human)
dy

In [ ]:
lp.get_virtual_reserve(eth)

In [ ]:
Lx = 100/(1/p_sqrt_human - 1/pb_sqrt_human)
Lx

In [ ]:
Ly = 1000/(p_sqrt_human - pa_sqrt_human)
Ly

In [ ]:
(Lx+Ly)/2

In [ ]:
lp.get_reserve(eth)

In [ ]:
63259.85082780032 - 63196.61638219618

In [ ]:
arb = CorrectReserves(lp, x0 = 1/exp_p_arr[0])
for k in range(1, n_steps):
    p = 1/exp_p_arr[k]
    arb.apply(p, lwr_tick, upr_tick)
    
    select_tkn = EventSelectionModel().bi_select(0.5)
    rnd_swap_amt = TokenDeltaModel(30).delta()
    rnd_add_amt = TokenDeltaModel(30).delta()
    user_add = random.choice(accounts)
    user_swap = random.choice(accounts)
    v3_positions = copy.deepcopy(lp.positions)
    if(select_tkn == 0): 
        AddLiquidity().apply(lp, eth, user_add, rnd_add_amt, lp.get_tick_price(-1), lp.get_tick_price(1))
        update_graph(lp, user_add, tkn, v3_positions, v3_graph)
        out = Swap().apply(lp, eth, user_swap, rnd_swap_amt) 
    else:
        AddLiquidity().apply(lp, tkn, user_add, p*rnd_add_amt, lp.get_tick_price(-1), lp.get_tick_price(1))
        update_graph(lp, user_add, tkn, v3_positions, v3_graph)
        out = Swap().apply(lp, tkn, user_swap,  p*rnd_swap_amt) 

    print(f'Market: {exp_p_arr[k]}, LP: {lp.get_price(tkn)}')

print('')
lp.summary()

In [ ]:
lp.ticks

In [ ]:
lp.get_tick_price(1)

In [ ]:
lp.ticks

In [ ]:
UniV3Utils.getPositionKey(accounts[0], lwr_tick, upr_tick)

In [ ]:
lp.positions

In [ ]:
#for pos in v3_graph:
#    print(f"price {v3_graph[pos]['price']}, liq: {v3_graph[pos]['delta_liq']/10**18}")

In [ ]:
for k, pos in enumerate(v3_graph):
    row = np.empty(2)
    row[0] = v3_graph[pos]['price']
    row[1] = v3_graph[pos]['delta_liq']/10**18
    res = row if k == 0 else np.vstack((res, row))

df = pd.DataFrame(res, columns = ['price', 'liq'])
df.sort_values(by=['price'], inplace=True)
df.reset_index(drop=True,inplace=True)
df['price'] = np.round(df['price'].values, 3)
freq_df_v3 = df.groupby('price').agg({'liq': 'sum'})

In [ ]:
fig, (price_ax, ld_ax) = plt.subplots(nrows=2, sharex=False, sharey=False, figsize=(12, 10))

price_ax.plot(x_val[1:], p_arr[1:], color = 'r',linestyle = 'dashdot', label='initial invest') 
price_ax.set_title(f'Price Chart (ETH/TKN)', fontsize=15)
price_ax.set_ylabel('Price (USD)', size=14)
price_ax.set_xlabel('Date', size=10)

ld_ax.axvline(x = lp.get_price(tkn), color = 'red', linestyle = 'dashdot', linewidth=2, label = 'Last Price')
ld_ax.bar(freq_df_v2.index, freq_df_v2['liq'].values, color ='purple', width = 0.00095, alpha = .5, label = 'Uni V2')
ld_ax.bar(freq_df_v3.index, freq_df_v3['liq'].values, color ='teal', width = 0.00095, alpha = .5, label = 'Uni V3')
ld_ax.set_title('Liquidity Frequency Distribution: Uniswap V2 vs V3',fontsize=15)
ld_ax.set_ylabel('Liquidity', fontsize=14)
ld_ax.set_xlabel('Price (USD)', fontsize=10)
ld_ax.legend()